In [1]:
import json
import numpy as np
import os

%load_ext autoreload
%autoreload 2

# K-Means anchors ratios

## Get COCO train 2017 annotations

In [2]:
ANNOTATIONS_PATH = "./annotations/instances_train2017.json"

!pip install tqdm
if not os.path.exists(ANNOTATIONS_PATH):
    !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
    !unzip annotations_trainval2017.zip

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
# efficientdet default anchors sizes
anchors_scales = [2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]
anchors_sizes = list(np.array([anchors_scale * np.array([32, 64, 128, 256, 512]) for anchors_scale in anchors_scales]).flatten())
anchors_sizes

[32.0,
 64.0,
 128.0,
 256.0,
 512.0,
 40.31747359663594,
 80.63494719327188,
 161.26989438654377,
 322.53978877308754,
 645.0795775461751,
 50.79683366298238,
 101.59366732596476,
 203.18733465192952,
 406.37466930385904,
 812.7493386077181]

## Get optimal anchors ratios using K-Means

In [4]:
from kmeans_anchors_ratios import get_optimal_anchors_ratios


anchors_ratios = get_optimal_anchors_ratios(
    ANNOTATIONS_PATH,
    input_size=512,
    normalizes_bboxes=True,
    num_runs=10,
    num_anchors_ratios=3,
    max_iter=300,
    min_size=0,
    iou_threshold=0.5,
    anchors_sizes=anchors_sizes,
    decimals=1,
)

2020-05-23 13:50:46 Starting the calculation of the optimal anchors ratios
2020-05-23 13:50:46 Reading annotations from ./annotations/instances_train2017.json
2020-05-23 13:51:06 Extracting and preprocessing bounding boxes
2020-05-23 13:51:11 Discarding 2 bounding boxes with size lower or equal to 0


2020-05-23 13:51:11 K-Means (10 runs): 100%|████████████| 10/10 [01:36<00:00,  9.64s/it]

2020-05-23 13:52:47 Best run avg. IoU: 80.48%
Runs avg. IoU: 80.48% ± 0.00% (mean ± std. dev. of 10 runs, 0 skipped)
2020-05-23 13:52:47 Avg. IoU between norm. anchors and bboxes: 80.48%


2020-05-23 13:52:47 Avg. IoU between bounding boxes and their most similar anchor: 62.63%
2020-05-23 13:52:58 Number of bounding boxes without similar anchors (IoU < 0.5):  239153/859999 (27.81%)
2020-05-23 13:52:58 Optimal anchors ratios: [(0.6, 1.5), (1.0, 1.0), (1.4, 0.7)]


## Get anchors from anchors ratios and sizes

In [5]:
from kmeans_anchors_ratios import get_anchors_from_ratios_and_sizes


anchors = get_anchors_from_ratios_and_sizes(anchors_ratios, anchors_sizes)
anchors

array([[  19.2       ,   48.        ],
       [  32.        ,   32.        ],
       [  44.8       ,   22.4       ],
       [  38.4       ,   96.        ],
       [  64.        ,   64.        ],
       [  89.6       ,   44.8       ],
       [  76.8       ,  192.        ],
       [ 128.        ,  128.        ],
       [ 179.2       ,   89.6       ],
       [ 153.6       ,  384.        ],
       [ 256.        ,  256.        ],
       [ 358.4       ,  179.2       ],
       [ 307.2       ,  768.        ],
       [ 512.        ,  512.        ],
       [ 716.8       ,  358.4       ],
       [  24.19048416,   60.47621039],
       [  40.3174736 ,   40.3174736 ],
       [  56.44446304,   28.22223152],
       [  48.38096832,  120.95242079],
       [  80.63494719,   80.63494719],
       [ 112.88892607,   56.44446304],
       [  96.76193663,  241.90484158],
       [ 161.26989439,  161.26989439],
       [ 225.77785214,  112.88892607],
       [ 193.52387326,  483.80968316],
       [ 322.53978877,  3

In [6]:
with open(ANNOTATIONS_PATH) as f:
    annotations = json.load(f)

In [7]:
from kmeans_anchors_ratios import get_annotations_without_similar_anchors

annotations_field = get_annotations_without_similar_anchors(
    annotations,
    anchors_ratios,
    anchors_sizes,
    input_size=512,
    iou_threshold=0.5,
    min_size=0,
)
bboxes_without_similar_anchors = [ann["bbox"] for ann in annotations_field]
# IMPORTANT: the bounding boxes are scaled according to the input_size before the comparison with the anchors but those returned are at original size
# e.g. if a bounding box has width = height = 40, input_size=512 and the original image size is 5120,
# the bounding box used to train the model is just 4x4 (and no anchor will have IoU > 50% with it)
print("First 5 bounding boxes without similar anchors:")
print(bboxes_without_similar_anchors[:5])

First 5 bounding boxes without similar anchors:
[[251.87, 333.42, 125.94, 22.71], [236.2, 146.01, 26.17, 19.49], [377.68, 258.81, 13.28, 9.36], [484.33, 224.38, 34.03, 22.54], [162.27, 189.06, 29.63, 25.24]]
